In [1]:
import re
import csv
import math
import uuid
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://www.amazon.in/HP-RX5500M-Graphics-Flicker-16-e0162AX/product-reviews/B098QBT5KT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews"
url = url + '&pageNumber=1'
headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0'}

column_header = ['unique id', 'name', 'rating', 'votes', 'title', 'location', 'date', 'genuinity', 'description']

In [3]:
def review_extractor(url, file_name, headers, column_header):
    source = requests.get(url, headers=headers)
    soup = BeautifulSoup(source.text, 'lxml')

    review_count = soup.find(class_="a-row a-spacing-base a-size-base").text
    pattern = r'[0-9]+'
    review_count = int(re.findall(pattern, review_count)[1])

    total_review_pages = math.ceil(review_count / 10)

    csv_file = open(r"reviews.csv", 'a', newline='', encoding='utf-8')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(column_header)

    loc_pat = r'[A-Z][a-z]+'
    date_pat = r'[0-9]{1,2}\s[A-Z][a-z]+\s[0-9]+'
    vote_pat = r'[0-9]+|[A-Z][a-z]+'

    print("Extracting:- ")
    for index, page in enumerate(range(total_review_pages), start=1):
        url_ = url[:-1] + str(index)        
        source = requests.get(url_, headers=headers).text
        soup = BeautifulSoup(source, 'lxml')
        print(url_)
        reviews = soup.find_all(class_="a-section celwidget")

        for review in reviews:
            name = review.find(class_='a-profile-name').text
            rating = review.find(class_='a-icon-alt').text
            title = review.find(class_="a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold").text
            date_place = review.find(class_="a-size-base a-color-secondary review-date").text
            loc = re.findall(loc_pat, date_place)[1]
            date = re.findall(date_pat, date_place)[0]
            try: genuine = review.find(class_="a-size-mini a-color-state a-text-bold").text
            except: genuine = ''
            try: 
                votes = review.find('span', class_="a-size-base a-color-tertiary cr-vote-text").text
                votes = re.findall(vote_pat, votes)[0]
            except: 
                votes = ''
            unique_id = uuid.uuid4().fields[2]
            desciption = review.find(class_="a-size-base review-text review-text-content").text
            variables = [unique_id, name, rating[:3], votes[:3], title.strip(), loc, date, genuine, desciption.strip()]
            csv_writer.writerow(variables)

    csv_file.close()
    return "Successfully extracted reviews to the csv file"

In [4]:
review_extractor(url, 'sample.csv', headers, column_header)

Extracting:- 
https://www.amazon.in/HP-RX5500M-Graphics-Flicker-16-e0162AX/product-reviews/B098QBT5KT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
https://www.amazon.in/HP-RX5500M-Graphics-Flicker-16-e0162AX/product-reviews/B098QBT5KT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
https://www.amazon.in/HP-RX5500M-Graphics-Flicker-16-e0162AX/product-reviews/B098QBT5KT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
https://www.amazon.in/HP-RX5500M-Graphics-Flicker-16-e0162AX/product-reviews/B098QBT5KT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
https://www.amazon.in/HP-RX5500M-Graphics-Flicker-16-e0162AX/product-reviews/B098QBT5KT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
https://www.amazon.in/HP-RX5500M-Graphics-Flicker-16-e0162AX/product-reviews/B098QBT5KT/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=6
https://www.am

'Successfully extracted reviews to the csv file'